In [1]:
import matplotlib.pyplot as plt

"""
        SETUP ENVIRONMENT

Choose a game:
- "Cart" # ok for 100 epoch
- "Car" # not good choice
- "Pendulum" # not good choice
- "Lake" # demand more then 1500 epoch
- "Maze" # 100 epoch is OK
- "Toy" # 100 epoch
"""
GAME_NAME = "Lake"
HORIZON = 10 #fixed horizon
"""
            INITIALIZING AGENTS and ENVIRONMENT
"""
"""-------------------------------------------------------------------"""
from enviroment import *
from utils import train_agent
import pandas as pd
env, obs_dim, action_dim = game_setup(GAME_NAME )
env.rnd_init_state = True
env.testing = True

ModuleNotFoundError: No module named 'gymnasium'

In [2]:
"""
                 SETUP AGENT and TRAINING
"""

"""REINFORCE"""
HIDDEN_DIM_REIN = 24
EPSILON = 0.1 # for epsilon greedy policy
"""MATRYOSHKA Hidden layer"""
HIDDEN_DIM_MTR = HIDDEN_DIM_REIN
"""ResNet"""
HIDDEN_DIM_RESNET = 10
#MTRNet with dynamically changing parameters number per layer
INIT_HIDDEN_LAYER = 20

"""Training"""
TAU = 0.9
LEARNING_RATE = 0.001
NUM_EPOCHES = 600
N_EPISODES = 25 # number of episodes per epoch.

"""Testing"""
NUM_TEST_EPISODES = 80


"""DYNAMICAL TAU AND LR"""
PATIENCE = 20 # for dynamical tau and lr
TAU_END = TAU # for dynamical tau
LR_END = LEARNING_RATE / 100 # for dynamical learning rate
"""
                    INITIALIZING AGENTS
"""
"""-------------------------------------------------------------------"""
#more settings---------------------
HORIZON_REIN = HORIZON
HORIZON_MTR = HORIZON
LEARNING_RATE_REIN = LEARNING_RATE
LEARNING_RATE_MTR  = LEARNING_RATE
LEARNING_RATE_RESNET = LEARNING_RATE
LEARNING_RATE_ORIGINAL = LEARNING_RATE
#-------------------------------------
HIDDEN_DIM_ORIGINAL = HIDDEN_DIM_RESNET
from original import OriginalMtrAgent
from NeuralNet import ReinforceAgent, MTRAgent
from MtrNet import ReinforceMtrNetAgent, MtrNetAgent, ShortLongAgent

#Reinforce Full Connected
agent1 = ReinforceAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_REIN,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_REIN, game_name=GAME_NAME)
agent1.policy.ntk_init(sigma_w=0,sigma_b=0)

#Reinforce MTRNet (Custom ResNet)
agent2 = ReinforceMtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU)
agent2.policy.ntk_init(sigma_w=0,sigma_b=0)

#MTR extra-dimension
agent3 = MTRAgent(obs_dim, action_dim, hidden_dim= HIDDEN_DIM_MTR , horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_MTR, game_name= GAME_NAME, tau=TAU)
agent3.policy.ntk_init(sigma_w=0,sigma_b=0)

#MTR with MTRNet
agent4 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU)
agent4.policy.ntk_init(sigma_w=0,sigma_b=0)

#Original MTR
agent5 = OriginalMtrAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_ORIGINAL,horizon= HORIZON, learning_rate= LEARNING_RATE_ORIGINAL , game_name= GAME_NAME, tau=TAU)
agent5.policy.ntk_init(sigma_w=0,sigma_b=0)

#MTR with MTRNet and dynamical number of parameter per layer
agent6 = MtrNetAgent(obs_dim,action_dim, hidden_dim=INIT_HIDDEN_LAYER, horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, dynamical_layer_param=True)
agent6.policy.ntk_init(sigma_w=0,sigma_b=0)

#ShortLongNet
agent7 = ShortLongAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU)
agent7.policy.ntk_init(sigma_w=0,sigma_b=0)

cuda


In [5]:
"""DataFrame to store results"""
FROM_PATH = "results/Lake.csv"
TO_PATH = "results/Lake.csv"
df = pd.read_csv(FROM_PATH)
data1 = {"init": "eoc", 'description': "Lake 8x8. One reward +30, each step reward -1, hole reward -5. Epoch: 600. Rein epsilon 0.15 "}

# function to push data
def push(df,data2,data3,data1=data1):
    data1.update(data2)
    data1.update(data3)
    df = df._append(data1, ignore_index=True)
    return df
#load to csv file path
def load(df,path=TO_PATH):
    df.to_csv(path, index=False)

In [58]:
"""
data = {
    'index': 0,
    'game': "Lol",
    'agent': "None",
    'init': "eoc",
    'lr': 0.00,
    'tau': 0.00,
    'test': 0.00,
    'train': 0.00,
    'full_rank': 0,
    'total_param': 0,
    'description': None,
}
df = df._append(data, ignore_index=True)
df = pd.DataFrame(data, index=[0])
df.to_csv("results.csv", index=False)
"""

'\ndata = {\n    \'index\': 0,\n    \'game\': "Lol",\n    \'agent\': "None",\n    \'init\': "eoc",\n    \'lr\': 0.00,\n    \'tau\': 0.00,\n    \'test\': 0.00,\n    \'train\': 0.00,\n    \'full_rank\': 0,\n    \'total_param\': 0,\n    \'description\': None,\n}\ndf = df._append(data, ignore_index=True)\ndf = pd.DataFrame(data, index=[0])\ndf.to_csv("results.csv", index=False)\n'

In [ ]:
"""
        TRAINING (REINFORCE CLASSIC)
"""
"""-------------------------------------------------------------------"""
"""Cart best for lr = 0.001"""
"""For Maze Zero init work bad, but eoc good"""
from utils import train_agent
epsilon = EPSILON
agent1.policy.ntk_init()
loss_list, entropy_loss_list, data = train_agent(agent1, env, num_epoches=NUM_EPOCHES, game_name=GAME_NAME, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, epsilon=epsilon, clip_grad=True)
"""Testing"""
from utils import test_agent
score, data_ = test_agent(agent1,env,GAME_NAME, n_episodes=NUM_TEST_EPISODES)
"""PUSH DATA"""
df = push(df, data, data_)

In [75]:
"""
    Matryoshka EXTRA-DIMENSION Training
"""
"""-------------------------------------------------------------------"""
agent3.tau = 0.5
agent3.lr = LEARNING_RATE
agent3.policy.ntk_init()
_,_,data3 = train_agent(agent3, env, num_epoches=3*NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent3.tau, lr_end=LR_END, patience=PATIENCE, clip_grad=False)
"""Testing"""
from utils import test_agent
_, data3_ = test_agent(agent3,env,GAME_NAME, n_episodes=NUM_TEST_EPISODES)
"""PUSH DATA"""
df =push(df, data3, data3_)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 51
tau 0.500
Learning rate 0.81677 * 10^-3
Reward: 0.28
Entropy Reward: -3.490720672607422

Epoch 101
tau 0.500
Learning rate 0.61297 * 10^-3
Reward: -4.76
Entropy Reward: -9.279319629669189

Epoch 151
tau 0.500
Learning rate 0.51154 * 10^-3
Reward: -0.84
Entropy Reward: -4.777075996398926

Epoch 201
tau 0.500
Learning rate 0.39579 * 10^-3
Reward: -0.84
Entropy Reward: -4.777075996398926

Epoch 251
tau 0.500
Learning rate 0.33671 * 10^-3
Reward: -0.84
Entropy Reward: -4.777075996398926

Epoch 301
tau 0.500
Learning rate 0.26772 * 10^-3
Reward: -0.28
Entropy Reward: -4.133898334503174

Epoch 351
tau 0.500
Learning rate 0.23171 * 10^-3
Reward: -3.08
Entropy Reward: -7.3497866439819335

Epoch 401
tau 0.500
Learning rate 0.18878 * 10^-3
Reward: -4.2
Entropy Reward: -8.636141967773437

Epoch 451
tau 0.500
Learning rate 0.16591 * 10^-3
Reward: -0.28
Entropy Reward: -4.133898334503174

Epoch 501
tau 0.500
Learning rate 0.13815 * 10^-3
Reward: -

C:\Users\koq1231\AppData\Local\Temp\ipykernel_17356\79008672.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(data1, ignore_index=True)


In [11]:
"""
    MTRNet Training
"""
"""-------------------------------------------------------------------"""
agent4.tau = 0.9
agent4.policy.ntk_init()
agent4.lightMTR = False
_,_,data4 = train_agent(agent4, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent4.tau , lr_end=LR_END, patience=PATIENCE, clip_grad=True, testing=False)
"""Testing"""
from utils import test_agent
_, data4_ = test_agent(agent4, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 51
tau 0.900
Learning rate 0.73942 * 10^-3
Reward: -2.96
Entropy Reward: -4.147830951288342

Epoch 101
tau 0.900
Learning rate 0.48771 * 10^-3
Reward: 16.16
Entropy Reward: 10.922254571914673

Epoch 151
tau 0.900
Learning rate 0.37800 * 10^-3
Reward: 17.44
Entropy Reward: 11.841959607601165

Epoch 201
tau 0.900
Learning rate 0.26604 * 10^-3
Reward: 20.84
Entropy Reward: 14.443393606543541

Epoch 251
tau 0.900
Learning rate 0.21455 * 10^-3
Reward: 23.76
Entropy Reward: 16.678531579971313

Epoch 301
tau 0.900
Learning rate 0.15950 * 10^-3
Reward: 23.64
Entropy Reward: 16.924810514450073

Epoch 351
tau 0.900
Learning rate 0.13302 * 10^-3
Reward: 23.76
Entropy Reward: 16.661598176956176

Epoch 401
tau 0.900
Learning rate 0.10357 * 10^-3
Reward: 23.68
Entropy Reward: 16.619761486053466

Epoch 451
tau 0.900
Learning rate 0.08885 * 10^-3
Reward: 23.76
Entropy Reward: 16.658895907402037

Epoch 501
tau 0.900
Learning rate 0.07193 * 10^-3
Reward: 

In [ ]:
data4_

In [12]:
"""PUSH DATA"""
df =push(df, data4, data4_)

In [ ]:
df.loc[df["agent"]=="MtrNet"]


In [33]:
"""
    MTRNet with dynamical number of parameters Training Dy
"""
"""-------------------------------------------------------------------"""
agent6.policy.ntk_init()
agent6.lr = LEARNING_RATE
agent6.tau = 0.7
_,_,data6 = train_agent(agent6, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent6.tau , lr_end=LR_END, patience=PATIENCE, clip_grad=True)
"""Testing"""
from utils import test_agent
_, data6_ = test_agent(agent6, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 51
tau 0.700
Learning rate 0.08333 * 10^-3
Reward: 8.0
Entropy Reward: 4.228945220510165

Epoch 101
tau 0.700
Learning rate 0.06425 * 10^-3
Reward: 7.333333333333333
Entropy Reward: 3.758174334963163

Epoch 151
tau 0.700
Learning rate 0.05448 * 10^-3
Reward: 9.333333333333334
Entropy Reward: 6.318525188912948

Epoch 201
tau 0.700
Learning rate 0.04307 * 10^-3
Reward: 9.666666666666666
Entropy Reward: 5.900981002797683

Epoch 251
tau 0.700
Learning rate 0.03711 * 10^-3
Reward: 8.666666666666666
Entropy Reward: 5.375679997106393

Epoch 301
tau 0.700
Learning rate 0.03000 * 10^-3
Reward: 9.0
Entropy Reward: 5.89129956625402

Epoch 351
tau 0.700
Learning rate 0.02622 * 10^-3
Reward: 8.333333333333334
Entropy Reward: 5.333344016224146

Epoch 401
tau 0.700
Learning rate 0.02164 * 10^-3
Reward: 8.666666666666666
Entropy Reward: 4.82641786026458

Epoch 451
tau 0.700
Learning rate 0.01916 * 10^-3
Reward: 7.666666666666667
Entropy Reward: 3.924909

In [34]:
"""PUSH DATA"""
df =push(df, data6, data6_)

C:\Users\koq1231\AppData\Local\Temp\ipykernel_17356\3362490508.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(data1, ignore_index=True)


In [35]:
"""
    Matryoshka "ORIGINAL" Training
"""

"""-------------------------------------------------------------------"""
agent5.policy.ntk_init()
agent5.tau = TAU
agent5.lr = LEARNING_RATE
_,_,data5 = train_agent(agent5, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent5.tau, lr_end=LR_END, patience=PATIENCE, clip_grad=False, testing=True)
"""Testing"""
from utils import test_agent
_, data5_ = test_agent(agent5, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 51
tau 0.700
Learning rate 0.08333 * 10^-3
Reward: 14.333333333333334
Entropy Reward: 13.641520530470492
Optimal entropy solution: None
CK full rank: None
Test reward: 20.55

Epoch 101
tau 0.700
Learning rate 0.06425 * 10^-3
Reward: 22.666666666666668
Entropy Reward: 20.327831843119686
Optimal entropy solution: None
CK full rank: None
Test reward: 18.775

Epoch 151
tau 0.700
Learning rate 0.05448 * 10^-3
Reward: 13.333333333333334
Entropy Reward: 11.750056453359624
Optimal entropy solution: None
CK full rank: None
Test reward: 19.7625

Epoch 201
tau 0.700
Learning rate 0.04307 * 10^-3
Reward: 12.0
Entropy Reward: 10.656896777373428
Optimal entropy solution: None
CK full rank: None
Test reward: 20.9625

Epoch 251
tau 0.700
Learning rate 0.03711 * 10^-3
Reward: 19.333333333333332
Entropy Reward: 17.669533358343568
Optimal entropy solution: None
CK full rank: None
Test reward: 19.1625

Epoch 301
tau 0.700
Learning rate 0.03000 * 10^-3
Rewar

In [ ]:
from utils import Ntk_analysis
ntk = Ntk_analysis(env)
ntk.full_analysis(agent5, mode="ntk")

In [ ]:
ntk.eigen_values

In [36]:
"""PUSH DATA"""
df =push(df, data5, data5_)

C:\Users\koq1231\AppData\Local\Temp\ipykernel_17356\3362490508.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(data1, ignore_index=True)


In [ ]:
df

In [ ]:

"""
    MTRNet Light Training
"""
"""-------------------------------------------------------------------"""
agent4.tau = TAU
agent4.lightMTR = True
agent4.lr = LEARNING_RATE
agent4.policy.ntk_init()
_,_,data9 = train_agent(agent4, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, clip_grad=False)

"""Testing"""
from utils import test_agent
_, data9_ = test_agent(agent4, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES)

In [ ]:
"""PUSH DATA"""
df =push(df, data9, data9_)

In [ ]:
"""
    ShortLongNet Training
    Remark: Horizon should be at least 10!
"""
"""-------------------------------------------------------------------"""
_,_,data7 = train_agent(agent7, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, clip_grad=False)
"""Testing"""
from utils import test_agent
_, data7_ = test_agent(agent7, env, GAME_NAME)
"""PUSH DATA"""
df =push(df, data7, data7_)

In [ ]:
"""
        TRAINING (REINFORCE WITH MTR NN)
        Remark: we are training two reinforce agents
"""
"""-------------------------------------------------------------------"""
agent2.lr = agent1.lr
from utils import train_agent
agent2.policy.ntk_init()
_, _, data2 = train_agent(agent2, env, num_epoches=NUM_EPOCHES, game_name=GAME_NAME, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END , patience=PATIENCE, clip_grad=False)
"""Testing"""
from utils import test_agent
_, data2_ = test_agent(agent2,env,GAME_NAME, n_episodes=NUM_TEST_EPISODES)
"""PUSH DATA"""
df =push(df, data2, data2_)

In [ ]:
"""Check the optimal solution"""
env.generate_all_q_stars(3, tau = 0.05)
env.v_star

In [ ]:
"""
        TEST AGENT (In render mode)
"""
agent = agent1
NUMBER_OF_EPISODES = 1
agent.tau = 0.05
#env.generate_all_q_stars(horizon = HORIZON, tau = agent.tau)
"""-------------------------------------------------------------------"""
from enviroment import render
render(agent,env, NUMBER_OF_EPISODES)


In [13]:
"""Write .csv file to save DataFrame"""
load(df)

In [ ]:
df